## Team Map

Pre requisitos:
Tabela com colunas: nome do colaborador, data da avaliação de proficiência(primeira avaliação), nível na avaliação de proficiência, data da última avaliação, nível na última avaliação.

In [63]:
# Nomes dos arquivos
NOME_TABELA_PRE = './indorama.xlsx'
NOME_TABELA_INICIAL = './indorama_inter.xlsx'
NOME_TABELA_TRANSFORMADA = "./indorama_transformed.xlsx"

# Nomes das colunas
NOME_COLUNA_NOME = 'usuario'
NOME_COLUNA_IDIOMA = 'Curso'
NOME_COLUNA_NIVEL_AVALIACAO = 'classificacao__ultimo_test'
NOME_COLUNA_NIVEL_PROFICIENCIA = 'classificacao_test_0'
NOME_COLUNA_DATA_AVALIACAO = 'data_ultimo_test'
NOME_COLUNA_DATA_PROFICIENCIA = 'data_test_0'
NOME_COLUNA_META_FINAL = 'Meta Final'
NOME_COLUNA_PRAZO_META = 'Prazo Meta final'
NOME_COLUNA_STATUS = 'Status'

# Valores de filtro
NOME_STATUS_ATIVO_FILTRO = 'Subsídio ativo'

## Imports

In [64]:
!which python

/home/pessoal/Documentos/gitc/jupyter-graphs-angela/myenv/bin/python


In [65]:
#requirement.txt
!pip install pandas
!pip install openpyxl
!pip install streamlit
!pip install plotly
!pip install matplotlib

In [66]:
import pandas as pd
import plotly.graph_objects as go
import streamlit
import plotly.express as px
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, HTML

# Table transformer
import openpyxl
import numpy as np
import os
import re
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Alignment, Border, Side, Font
from dateutil.relativedelta import relativedelta

In [67]:
nomes = [
    "Adriana Gomes Silva",
    "Adriano Frugieri De Oliveira",
    "Aline Da Silva Chelan",
    "Carlos Afonso De Amorim",
    "Cleber Barbosa Da Silva",
    "Eduardo Osti Marinho De Moura",
    "Fabiana Braz Da Silva",
    "Fagner Virgens Silva",
    "Gildson Rocha Santos",
    "Gizeli Silva",
    "Ismael Pereira De Jezus",
    "Janete Rodrigues Pereira Da Silva",
    "Jeferson Marques Goncalves",
    "Juliana Siqueira Da Cruz",
    "Lais De Freitas Fernandes Benvenga",
    "Leonardo Barreto Da Silva",
    "Luciana Cristina Anselmo",
    "Luiz Henrique De Souza",
    "Marcelo Alves Dos Reis",
    "Marcos Roberto Pedro",
    "Mayara Aparecida Rocha Garcia",
    "Paulo Augusto Batista Junior",
    "Priscila Nolberto Fernandes De Alencar",
    "Priscilla Fernandes Yamaguchi",
    "Rafael Tezolin Ribeiro Da Silva",
    "Raquel Ferreira Burato",
    "Roberta Nunes Maximo",
    "Rodrigo Assman",
    "Ronaldo Marui",
    "Saulo Andrade De Oliveira",
    "Thamyris Do Nascimento Gabriel",
    "Vanessa Souto Breder",
    "Washington Leao Machado"
]

# Transformers

In [68]:
import pandas as pd
from datetime import datetime

df = pd.read_excel(NOME_TABELA_PRE) 
# Converter a coluna 'data_conclusao_do_teste' para o tipo datetime
df['data_conclusao_do_teste'] = pd.to_datetime(df['data_conclusao_do_teste'])

# Ordenar o DataFrame por 'usuario' e 'data_conclusao_do_teste'
df = df.sort_values(by=['usuario', 'data_conclusao_do_teste'])

# Criar uma lista para armazenar os novos registros
novos_registros = []

# Agrupar o DataFrame por 'usuario'
grupos_usuario = df.groupby('usuario')
data_atual = datetime.now()

# Iterar sobre cada grupo de usuário
for usuario, grupo_usuario in grupos_usuario:
    if usuario not in nomes:
        continue
    # Dentro de cada usuário, agrupar por 'idioma'
    grupos_idioma = grupo_usuario.groupby('natureza')
    
    for idioma, grupo_idioma in grupos_idioma:
        grupo_idioma = grupo_idioma.reset_index(drop=True)
        
        # Criar um novo registro para cada combinação de usuário e idioma
        novo_registro = {
            NOME_COLUNA_NOME: usuario,
            NOME_COLUNA_META_FINAL: grupo_idioma.iloc[-1]['meta_estabelecida'],  # Meta da última prova
            NOME_COLUNA_PRAZO_META: grupo_idioma.iloc[-1]['prazo_meta_estabelecida'],  # Prazo da última prova
            NOME_COLUNA_IDIOMA: idioma,  # Idioma específico
            NOME_COLUNA_NIVEL_AVALIACAO: grupo_idioma.iloc[-1]['classificacao'],
            NOME_COLUNA_DATA_AVALIACAO: grupo_idioma.iloc[-1]['data_conclusao_do_teste'],
        }
        
        # Encontrar o índice onde 'objetivo_do_teste' é 'Avaliação de proficiência'
        x = 0
        i = -1
        for gp in grupo_idioma.iloc:
            i += 1
            if gp['objetivo_do_teste'] == 'Avaliação de proficiência':
                x = i
            
        
        # Adicionar as datas e classificações dos testes a partir do índice x
        for i, gp in grupo_idioma.iloc[x:].iterrows():
            novo_registro[f"data_test_{i}"] = gp['data_conclusao_do_teste']
            novo_registro[f"classificacao_test_{i}"] = gp['classificacao']
        
        # Adicionar o novo registro à lista
        novos_registros.append(novo_registro)

# Criar um novo DataFrame com os novos registros
nova_tabela = pd.DataFrame(novos_registros)

# Opcional: salvar a nova tabela em um arquivo Excel
nova_tabela.to_excel(NOME_TABELA_INICIAL, index=False)

print("Nova tabela criada com sucesso!")


Nova tabela criada com sucesso!


In [69]:
import pandas as pd
import numpy as np
import os
from openpyxl.styles import PatternFill, Alignment, Border, Side, Font

# Carregar os dados do arquivo
data = pd.read_excel(NOME_TABELA_INICIAL)

# Identificar todas as colunas de classificação e data
classification_columns = [col for col in data.columns if 'classificacao_test_' in col]
date_columns = [col for col in data.columns if 'data_test_' in col]

# Garantir que as colunas estejam ordenadas corretamente
classification_columns.sort(key=lambda x: int(x.split('_')[-1]))
date_columns.sort(key=lambda x: int(x.split('_')[-1]))

print(classification_columns)
print(date_columns)

# Verificar e substituir 'Ground zero' por 'Marco zero' nas colunas de classificações
for col in classification_columns:
    data[col] = data[col].replace('Ground zero', 'Marco zero')
    data[col] = data[col].replace('Marco cero', 'Marco zero')

# Ordenar os dados pelo nome do colaborador
if NOME_COLUNA_NOME in data.columns:
    data = data.sort_values(by=NOME_COLUNA_NOME)
else:
    raise KeyError(f"A coluna '{NOME_COLUNA_NOME}' não foi encontrada no arquivo Excel.")

# Lista de idiomas
idiomas = data[NOME_COLUNA_IDIOMA].unique()
tabelas_classificacoes = {}

# Criar lista de classificações para cada idioma
for idioma in idiomas:
    classificacoes_idioma = []
    data_filtrada_idioma = data[data[NOME_COLUNA_IDIOMA] == idioma]
    for col in classification_columns:
        classificacoes_idioma += data_filtrada_idioma[col].unique().tolist()
    classificacoes_idioma += data_filtrada_idioma[NOME_COLUNA_META_FINAL].unique().tolist()
    # Remover duplicatas e valores nulos
    classificacoes_idioma = [x for x in set(classificacoes_idioma) if pd.notna(x) and x != ""]
    tabelas_classificacoes[idioma] = classificacoes_idioma

# Função para ordenar as classificações
def ordenar_classificacoes(classificacoes):
    def classificacao_key(nivel):
        if isinstance(nivel, str) and (nivel.lower() == 'marco zero' or nivel.lower() == 'sem definição' or nivel.lower() == 'n/a'):
            return (0, 0, 0)
        elif isinstance(nivel, str):
            if '.' not in nivel:
                letra = nivel
                return (ord(letra[0].upper()) - ord('A'), int(letra[1]), 0)
            nivel_split = nivel.split('.')
            letra = nivel_split[0]
            plus = 0
            if '+' in letra:
                letra = letra[:-1]
                plus = 0.1
            numero = int(nivel_split[1])
            return (ord(letra[0].upper()) - ord('A'), int(letra[1]) + plus, numero)
        return (float('inf'), 0, 0)
    return sorted([x for x in classificacoes if pd.notna(x) and x != ""], key=classificacao_key)

# Função para preencher a tabela transformada
def preencher_tabela_transformada(row, classificacoes):
    linha_usuario = {nivel: np.nan for nivel in classificacoes}
    # Coletar todas as classificações e datas
    classificacoes_datas = []
    # Adicionar classificações de testes
    for class_col, date_col in zip(classification_columns, date_columns):
        classificacao = row.get(class_col, None)
        data_teste = row.get(date_col, None)
        if pd.notna(classificacao) and pd.notna(data_teste) and classificacao in classificacoes:
            data_teste = pd.to_datetime(data_teste).strftime('%m/%Y')
            classificacoes_datas.append((classificacao, data_teste))            
    # Ordenar as classificações por data
    classificacoes_datas.sort(key=lambda x: pd.to_datetime(x[1], format='%m/%Y'))
    # Preencher as classificações com as datas
    for classificacao, data in classificacoes_datas:
        linha_usuario[classificacao] = data
    # Preencher 'Caminho Percorrido' entre as classificações
    indices = [classificacoes.index(c) for c, d in classificacoes_datas if c in classificacoes]
    for i in range(len(indices) - 1):
        start = indices[i]
        end = indices[i+1]
        for idx in range(min(start, end)+1, max(start, end)):
            nivel = classificacoes[idx]
            if linha_usuario[nivel] is np.nan:
                linha_usuario[nivel] = 'Caminho Percorrido'
    #print(classificacao, data_teste)
    # Adicionar meta final
    meta_final = row.get(NOME_COLUNA_META_FINAL, None)
    prazo_meta = row.get(NOME_COLUNA_PRAZO_META, None)
    print("Meta_final:", meta_final)
    print("Prazo_meta:", prazo_meta)
    if pd.notna(prazo_meta):
        prazo_meta = pd.to_datetime(prazo_meta).strftime('%m/%Y')
    if meta_final and pd.notna(meta_final) and meta_final in classificacoes:
        linha_usuario[meta_final] = prazo_meta
    # Adicionar 'Caminho a Percorrer' até a meta final
    if classificacoes_datas and meta_final and meta_final in classificacoes:
        last_classificacao = classificacoes_datas[-1][0]
        if last_classificacao in classificacoes:
            start_idx = classificacoes.index(last_classificacao)
            end_idx = classificacoes.index(meta_final)
            for idx in range(min(start_idx, end_idx)+1, max(start_idx, end_idx)):
                nivel = classificacoes[idx]
                if linha_usuario[nivel] is np.nan:
                    linha_usuario[nivel] = 'Caminho a Percorrer'
    return pd.Series(linha_usuario)

# Deletar o arquivo existente, se houver
if os.path.exists(NOME_TABELA_TRANSFORMADA):
    os.remove(NOME_TABELA_TRANSFORMADA)

# Criar tabelas separadas por idioma e salvar em Excel
with pd.ExcelWriter(NOME_TABELA_TRANSFORMADA, engine='openpyxl') as writer:
    for idioma in idiomas:
        classificacoes = ordenar_classificacoes(tabelas_classificacoes[idioma])
        data_filtrada = data[data[NOME_COLUNA_IDIOMA] == idioma]
        tabela_transformada = data_filtrada.apply(preencher_tabela_transformada, axis=1, classificacoes=classificacoes)
        usuarios = data_filtrada[NOME_COLUNA_NOME]
        tabela_transformada.index = usuarios
        tabela_transformada.to_excel(writer, sheet_name=idioma)

        # Ajustar tamanhos das células e aplicar cores
        workbook = writer.book
        worksheet = writer.sheets[idioma]

        # Garantir que a planilha esteja visível
        worksheet.sheet_state = 'visible'

        # Ajustar largura das colunas
        for col in worksheet.columns:
            max_length = 0
            column = col[0].column_letter  # Coluna
            for cell in col:
                try:
                    if cell.value is not None:
                        cell_length = len(str(cell.value))
                        if cell_length > max_length:
                            max_length = cell_length
                except:
                    pass
            adjusted_width = (max_length + 2)
            worksheet.column_dimensions[column].width = adjusted_width

        # Aplicar cores ao cabeçalho e células
        header_fill = PatternFill(start_color='1f2f36', end_color='1f2f36', fill_type='solid')
        cell_fill = PatternFill(start_color='391e70', end_color='391e70', fill_type='solid')
        cell_fill_meta_final = PatternFill(start_color='adc22f', end_color='adc22f', fill_type='solid')
        cell_fill_caminho_a_percorrer = PatternFill(start_color='adc22f', end_color='adc22f', fill_type='solid')
        header_alignment = Alignment(horizontal='center', vertical='center')
        cell_alignment = Alignment(horizontal='center', vertical='center')
        thin_border = Border(left=Side(style='thin'), right=Side(style='thin'),
                             top=Side(style='thin'), bottom=Side(style='thin'))
        header_font = Font(color='FFFFFF')

        # Colorir cabeçalho
        for cell in worksheet[1]:
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.font = header_font  # Texto branco
            cell.border = thin_border

        # Colorir primeira coluna
        for cell in worksheet['A']:
            if cell.row > 1:
                cell.fill = header_fill
                cell.font = header_font
                cell.alignment = cell_alignment
                cell.border = thin_border

        # Colorir células preenchidas, o caminho percorrido, e aplicar bordas e alinhamento
        for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row,
                                       min_col=2, max_col=worksheet.max_column):
            aux = 0
            for cell in row:
                if cell.value not in ['Caminho Percorrido', 'Caminho a Percorrer', ""]:
                    aux += 1

            aux = 0
            for cell in row:
                if cell.value == 'Caminho Percorrido':
                    cell.fill = cell_fill
                    cell.value = ""
                elif cell.value == 'Caminho a Percorrer':
                    cell.fill = cell_fill_caminho_a_percorrer
                    cell.value = ""
                    aux = 2
                elif cell.value is not None and cell.value != "":
                    cell.fill = cell_fill
                    aux += 1
                    if aux == 3:
                        cell.fill = cell_fill_meta_final
                    else:
                        cell.font = header_font
                    cell.border = thin_border
                cell.alignment = cell_alignment

        # Fixar primeira linha e primeira coluna
        worksheet.freeze_panes = 'B2'

print("Tabela salva com sucesso!")

['classificacao_test_0', 'classificacao_test_1', 'classificacao_test_2']
['data_test_0', 'data_test_1', 'data_test_2']
Meta_final: B2.2
Prazo_meta: 2024-12-01 00:00:00
Meta_final: B1.2
Prazo_meta: 2026-01-01 00:00:00
Meta_final: B1.1
Prazo_meta: 2024-12-01 00:00:00
Meta_final: A1.2
Prazo_meta: 2024-12-01 00:00:00
Meta_final: B2.1
Prazo_meta: 2025-07-01 00:00:00
Meta_final: A2.2
Prazo_meta: 2024-12-01 00:00:00
Meta_final: A1.2
Prazo_meta: 2024-12-01 00:00:00
Meta_final: A1.2
Prazo_meta: 2024-12-01 00:00:00
Meta_final: B2+.1
Prazo_meta: 2024-12-01 00:00:00
Meta_final: A2.2
Prazo_meta: 2024-12-01 00:00:00
Meta_final: B2.2
Prazo_meta: 2024-12-01 00:00:00
Meta_final: B2.2
Prazo_meta: 2024-12-01 00:00:00
Meta_final: A2.2
Prazo_meta: 2024-12-01 00:00:00
Meta_final: B2+.1
Prazo_meta: 2024-12-01 00:00:00
Meta_final: B1.2
Prazo_meta: 2024-12-01 00:00:00
Tabela salva com sucesso!


In [70]:
df = pd.read_excel(NOME_TABELA_TRANSFORMADA)
df.fillna("", inplace=True)
df.head(5)

,usuario,Marco zero,A1.2,A2.1,A2.2,B1.1,B1.2,B2.1,B2.2,B2+.1
0,Adriana Gomes Silva,,,,,,12/2023,,12/2024,
1,Adriano Frugieri De Oliveira,,12/2022,,12/2023,,01/2026,,,
2,Aline Da Silva Chelan,,,12/2023,,12/2024,,,,
3,Gildson Rocha Santos,12/2023,12/2024,,,,,,,
4,Gizeli Silva,,,,,05/2024,,07/2025,,


# TeamMap

## Distribuição de pessoas por nivel de proficiência

In [71]:
# Definir o renderizador para um compatível com interações
pio.renderers.default = "iframe"  

data_original = pd.read_excel(NOME_TABELA_INICIAL, sheet_name=0)
# Carregar dados

# Renomear a coluna para facilitar o acesso
data_original.rename(columns={NOME_COLUNA_NIVEL_AVALIACAO: 'Proficiency'}, inplace=True)
color_scale = [
    "#1f77b4",  # Azul escuro
    "#337ebc",
    "#4c8eb3",
    "#6699aa",
    "#80a0a1",
    "#99a799",
    "#b3ae90",
    "#cca588",
    "#e6b07f",
    "#ffba76",
    "#ffb36d",
    "#ffad63",
    "#ffa559",
    "#ff9f4f",
    "#ff9945",
    "#ff933b",
    "#ff8d31",
    "#ff8727",
    "#ff811d",
    "#ff7b13",
    "#ff7509",
    "#ff6f00"   # Vermelho
]

# Substituir 'Ground zero' por 'Marco zero'
if('Proficiency' not in data_original.columns):
    exit(0)

data_original['Proficiency'] = data_original['Proficiency'].replace('Ground zero', 'Marco zero')
data_original['Proficiency'] = data_original['Proficiency'].replace('Marco cero', 'Marco zero')

# Definir a ordem dos níveis do CEFR
niveis_cefr = [
    "Marco zero", "A1.1", "A1.2", "A2.1", "A2.2", 
    "B1.1", "B1.2", "B2.1", "B2.2", "B2+.1", 
    "B2+.2", "C1.1", "C1.2", "C1.3", "C2"
]

# Contar ocorrências de cada nível do CEFR
contagem_proficiencia = data_original['Proficiency'].value_counts().reindex(niveis_cefr, fill_value=0)

# Calcular porcentagens e adicionar o símbolo %
porcentagem_proficiencia = (contagem_proficiencia / contagem_proficiencia.sum() * 100).round(2).astype(str) + '%'

# Preparar os dados para plotagem
dados_proficiencia = pd.DataFrame({
    "Nível de Proficiência": niveis_cefr,
    "Contagem": contagem_proficiencia.values,
    "Porcentagem": porcentagem_proficiencia.values
})

# Criar um gráfico de barras com uma escala de cores discreta, atualizando o texto para incluir as porcentagens diretamente nas barras
fig = px.bar(
    dados_proficiencia,
    x="Nível de Proficiência",
    y="Contagem",
    title="Distribuição dos Níveis de Proficiência - Última avaliação",
    labels={"Contagem": "Número de Pessoas", "Nível de Proficiência": "Níveis do CEFR"},
    text="Porcentagem",  # Mostrar a porcentagem diretamente em cada barra
    color="Nível de Proficiência",  # Adicionar diferenciação de cor
    color_discrete_sequence=color_scale  # Usar uma escala de cores qualitativa
)

# Melhorar o layout e a estética
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis=dict(categoryorder='array', categoryarray=niveis_cefr),
    template="plotly_white",  # Tema claro para maior destaque
    title_font=dict(size=20, family='Arial, bold'),
    xaxis_title_font=dict(size=14, family='Arial'),
    yaxis_title_font=dict(size=14, family='Arial'),
)

# Mostrar o gráfico
fig.show()
fig.write_html(NOME_TABELA_INICIAL + '_barras_HTML.html', auto_open=True)


## Mapeamento individual

In [72]:
# 1. Ler a planilha específica
transformed_single_language = pd.read_excel(NOME_TABELA_TRANSFORMADA, sheet_name='idioma inglês')

# 2. Verificar se a coluna NOME_COLUNA_NOME existe
if NOME_COLUNA_NOME not in transformed_single_language.columns:
    raise KeyError(f"A coluna '{NOME_COLUNA_NOME}' não está presente no DataFrame")

# 3. Definir as colunas CEFR relevantes
cef_columns = ['Marco zero', 'A1.1', 'A1.2', 'A2.1', 'A2.2', 
              'B1.1', 'B1.2', 'B2.1', 'B2.2', 'B2+.1', 
              'B2+.2', 'C1.1', 'C1.2', 'C1.3']
cef_columns = [col for col in cef_columns if col in transformed_single_language.columns]
# 4. Selecionar apenas as colunas existentes
transformed_single_language = transformed_single_language[
    [NOME_COLUNA_NOME] + cef_columns
]

# 5. Extrair as colunas de colaboradores e datas
colaboradores = transformed_single_language[[NOME_COLUNA_NOME]]
dates = transformed_single_language[cef_columns]

# 6. Converter os dados para formato longo (long format)
long_data = pd.melt(
    pd.concat([colaboradores, dates], axis=1),
    id_vars=[NOME_COLUNA_NOME],
    var_name='CEFR Level',
    value_name='Date'
)

# 7. Remover linhas com datas NaN
long_data = long_data.dropna(subset=['Date'])

# 8. Definir a ordem dos níveis CEFR, incluindo "Marco zero"
# 9. Converter 'CEFR Level' para categórico ordenado
long_data['CEFR Level'] = pd.Categorical(
    long_data['CEFR Level'],
    categories=cef_columns,
    ordered=True
)

# 10. Ordenar os dados pelo colaborador e pela ordem dos níveis CEFR
long_data = long_data.sort_values(by=[NOME_COLUNA_NOME, 'CEFR Level'])

# 11. Criar figura
fig = go.Figure()

# 12. Preparar um dicionário para mapear as cores dos marcadores por indivíduo
marker_color_map = {}

# 13. Adicionar linhas para cada indivíduo
for name in long_data[NOME_COLUNA_NOME].unique():
    individual_data = long_data[long_data[NOME_COLUNA_NOME] == name].sort_values(by='CEFR Level')
    
    # Definir as cores das linhas com base na lógica original
    if len(individual_data) == 2:
        line_segments = ['#adc22f']
        marker_colors = ['#391e70', '#adc22f']
    elif len(individual_data) >= 3:
        # Ajuste para mais de 2 pontos
        line_segments = ['#391e70'] * (len(individual_data) - 2) + ['#adc22f']
        marker_colors = ['#391e70'] * (len(individual_data) - 1) + ['#adc22f']
    else:
        # Apenas um ponto
        line_segments = []
        marker_colors = ['#391e70']
    
    # Adicionar segmentos de linhas conectando apenas pontos consecutivos
    for i in range(len(individual_data) - 1):
        fig.add_trace(
            go.Scatter(
                x=individual_data['CEFR Level'].iloc[i:i+2],
                y=[name, name],
                mode='lines',
                line=dict(width=2, color=line_segments[i] if i < len(line_segments) else '#391e70'),
                showlegend=False,
            )
        )
    
    # Armazenar as cores dos marcadores
    marker_color_map[name] = marker_colors

# 14. Adicionar marcadores após as linhas para garantir que fiquem por cima
for name in long_data[NOME_COLUNA_NOME].unique():
    individual_data = long_data[long_data[NOME_COLUNA_NOME] == name].sort_values(by='CEFR Level')
    colors = marker_color_map.get(name, ['#391e70'] * len(individual_data))
    
    fig.add_trace(
        go.Scatter(
            x=individual_data['CEFR Level'],
            y=[name] * len(individual_data),
            mode='markers',
            marker=dict(size=8, color=colors),
            text=individual_data['Date'],
            hoverinfo='text',
            showlegend=False,
        )
    )

# 15. Ordenar os membros da equipe
team_members = sorted(long_data[NOME_COLUNA_NOME].unique())

# 16. Atualizar layout da figura com a ordem correta das categorias
fig.update_layout(
    title='Mapa da Equipe e Linha do Tempo da Progressão dos Níveis CEFR',
    xaxis_title='Nível CEFR',
    yaxis_title='Membros da Equipe',
    xaxis=dict(
        showgrid=True,
        fixedrange=False,  # Permitir zoom e pan
        type='category',
        categoryorder='array',
        categoryarray=cef_columns,  # Ordem especificada dos níveis CEFR
        range=['Marco zero', 'C1.3'],  # Ajuste conforme necessário
        side='top'
    ),
    yaxis=dict(
        showgrid=False,
        automargin=True,
        autorange='reversed',
        fixedrange=False,  # Permitir zoom e pan
        type='category',
        categoryorder='array',
        categoryarray=team_members,  # Ordem alfabética dos membros
    ),
    template='plotly_white',
    height=800,
    width=1000,
    margin=dict(l=50, r=50, t=50, b=50),
)

# 17. Adicionar anotações para as datas nos marcadores
for _, row in long_data.iterrows():
    fig.add_annotation(
        x=row['CEFR Level'],
        y=row[NOME_COLUNA_NOME],
        text=row['Date'],
        showarrow=False,
        font=dict(size=10),
        align='center',
        xanchor='center',
        yshift=-10,  # Mover texto para baixo
        xshift=-20,  # Mover texto para a esquerda
    )

# 18. Exibir o gráfico
fig.show()
fig.write_html(NOME_TABELA_INICIAL + '_META_HTML.html', auto_open=True)

## Distribuição de Colaboradores por Nível CEFR

In [73]:
niveis_cefr = [
    "Marco zero", "A1.1", "A1.2", "A2.1", "A2.2", 
    "B1.1", "B1.2", "B2.1", "B2.2", "B2+.1", 
    "B2+.2", "C1.1", "C1.2", "C1.3", "C2"
]

# Optional: Define the order for 'Grupo Nível'
grupo_niveis_order = ['Nível limitado', 'Nível independente', 'Nível sólido', 'Nível competente']

# Step 2: Display Custom CSS (Optional)
display(HTML("""
    <style>
        .output_scroll { height: auto !important; max-height: none !important; }
    </style>
"""))
# Step 4: Load the Excel File
try:
    df = pd.read_excel(NOME_TABELA_INICIAL)
except FileNotFoundError:
    raise FileNotFoundError(f"The file {NOME_TABELA_INICIAL} was not found. Please check the path.")

# Step 5: Rename Columns for Consistency
df.rename(columns={
    NOME_COLUNA_NOME: 'Colaborador',
    NOME_COLUNA_NIVEL_PROFICIENCIA: 'Classificação'
}, inplace=True)

# Step 6: Define the Grouping Function
def group_level(level):
    """
    Groups the CEFR levels into broader categories.

    Parameters:
    - level (str): The CEFR level of a collaborator.

    Returns:
    - str: The grouped level category.
    """
    if isinstance(level, str):
        if level.startswith('A') or level.startswith('M') or level.startswith('G'):
            return 'Nível limitado'
        elif level.startswith('B'):
            if '+' in level:
                return 'Nível sólido'
            return 'Nível independente'
        elif level.startswith('C'):
            return 'Nível competente'
    return None  # Exclude 'Outro' and any non-matching entries

# Step 7: Apply the Grouping Function
df['Grupo Nível'] = df['Classificação'].apply(group_level)

# Step 8: Filter Out 'Outro' and Any Non-Matching Entries
df_filtered = df.dropna(subset=['Grupo Nível']).copy()

# Step 9: Get Unique 'Idioma' Values
idiomas = df_filtered[NOME_COLUNA_IDIOMA].dropna().unique()
idiomas_sorted = sorted(idiomas)  # Optional: sort the languages alphabetically

# Step 10: Define the Function to Create Sunburst Chart for a Given 'Idioma'
def create_sunburst(selected_idioma):
    """
    Creates a Sunburst chart for the selected 'Idioma'.

    Parameters:
    - selected_idioma (str): The selected language.

    Returns:
    - Plotly Figure: The Sunburst chart.
    """
    # Filter data for the selected 'Idioma'
    df_idioma = df_filtered[df_filtered[NOME_COLUNA_IDIOMA] == selected_idioma].copy()

    # Step 11: Count the Number of Collaborators in Each CEFR Level
    cefr_counts = df_idioma['Classificação'].value_counts().reset_index()
    cefr_counts.columns = ['Classificação', 'Count']

    # Set 'Classificação' as a categorical variable with the specified order
    cefr_counts['Classificação'] = pd.Categorical(
        cefr_counts['Classificação'],
        categories=niveis_cefr,
        ordered=True
    )

    # Step 12: Assign 'Grupo Nível' to Each CEFR Level
    cefr_counts['Grupo Nível'] = cefr_counts['Classificação'].apply(group_level)

    # Set 'Grupo Nível' as a categorical variable with the specified order
    cefr_counts['Grupo Nível'] = pd.Categorical(
        cefr_counts['Grupo Nível'],
        categories=grupo_niveis_order,
        ordered=True
    )

    # Drop any rows where 'Grupo Nível' is NaN after categorization
    cefr_counts.dropna(subset=['Grupo Nível'], inplace=True)

    # Calculate percentages
    total_count = cefr_counts['Count'].sum()
    cefr_counts['Percentage'] = (cefr_counts['Count'] / total_count) * 100

    # Sort the DataFrame to respect the specified order
    cefr_counts.sort_values(['Grupo Nível', 'Classificação'], inplace=True)

    # Step 13: Create the Sunburst Chart
    fig = px.sunburst(
        cefr_counts,
        path=['Grupo Nível', 'Classificação'],
        values='Count',
        color='Grupo Nível',
        color_discrete_map={
            'Nível limitado': 'lightblue',
            'Nível independente': 'lightgreen',
            'Nível competente': 'lightcoral',
            'Nível sólido': 'lightgray'
        },
        title=f'Distribuição de Colaboradores por Nível CEFR - Idioma: {selected_idioma} - Proficiência',
        hover_data={'Count': True, 'Percentage': ':.2f'},  # Add percentage to hover data
        labels={'Count': 'Número de Colaboradores', 'Percentage': 'Porcentagem'}
    )

    # Step 14: Update Layout for Better Appearance
    fig.update_layout(
        margin=dict(t=50, l=25, r=25, b=25)
    )

    return fig

# Step 15: Create an Interactive Dropdown Widget
idioma_dropdown = widgets.Dropdown(
    options=idiomas_sorted,
    value=idiomas_sorted[0] if len(idiomas_sorted) > 0 else None,  # Set the default value to the first language
    description='Idiomas:',
    disabled=False,
)

# Step 16: Define the Function to Update the Plot Based on Dropdown Selection
def update_plot(selected_idioma):
    if selected_idioma is not None:
        fig = create_sunburst(selected_idioma)
        fig.show()
        fig.write_html(NOME_TABELA_INICIAL + '_PIZZA_HTML.html', auto_open=True)
    else:
        print("No idioma selected.")

# Step 17: Use `interactive_output` to Link the Dropdown with the Plot Update Function
out = widgets.interactive_output(update_plot, {'selected_idioma': idioma_dropdown})

# Step 18: Display the Dropdown and the Output Plot
display(idioma_dropdown, out)


Dropdown(description='Idiomas:', options=('idioma inglês',), value='idioma inglês')

Output()

In [74]:
# Step 1: Adjust Jupyter Notebook Output Cell Height
display(HTML("""
    <style>
        .output_scroll { height: auto !important; max-height: none !important; }
    </style>
"""))

# Step 2: Load the Excel File
df = pd.read_excel(NOME_TABELA_INICIAL)

# Step 3: Rename Columns for Consistency
df.rename(columns={
    NOME_COLUNA_NOME: NOME_COLUNA_NOME,
    NOME_COLUNA_NIVEL_AVALIACAO: 'Classificação'
}, inplace=True)

# Step 4: Define the Grouping Function
def group_level(level):
    """
    Groups the CEFR levels into broader categories.

    Parameters:
    - level (str): The CEFR level of a collaborator.

    Returns:
    - str: The grouped level category.
    """
    if isinstance(level, str):
        if level.startswith('A') or level.startswith('M') or level.startswith('G'):
            return 'Nível limitado'
        elif level.startswith('B'):
            if '+' in level:
                return 'Nível sólido'
            return 'Nível independente'
        elif level.startswith('C'):
            return 'Nível competente'
    return None  # Exclude 'Outro' and any non-matching entries

# Step 5: Apply the Grouping Function
df['Grupo Nível'] = df['Classificação'].apply(group_level)

# Step 6: Filter Out 'Outro' and Any Non-Matching Entries
df_filtered = df.dropna(subset=['Grupo Nível']).copy()

# Step 7: Get Unique 'Idioma' Values
idiomas = df_filtered[NOME_COLUNA_IDIOMA].dropna().unique()
idiomas_sorted = sorted(idiomas)  # Optional: sort the languages alphabetically

# Step 8: Define the Function to Create Sunburst Chart for a Given 'Idioma'
def create_sunburst(selected_idioma):
    """
    Creates a Sunburst chart for the selected 'Idioma'.

    Parameters:
    - selected_idioma (str): The selected language.

    Returns:
    - Plotly Figure: The Sunburst chart.
    """
    # Filter data for the selected 'Idioma'
    df_idioma = df_filtered[df_filtered[NOME_COLUNA_IDIOMA] == selected_idioma].copy()

    # Step 9: Count the Number of Collaborators in Each CEFR Level
    cefr_counts = df_idioma['Classificação'].value_counts().reset_index()
    cefr_counts.columns = ['Classificação', 'Count']

    # Step 10: Assign 'Grupo Nível' to Each CEFR Level
    cefr_counts['Grupo Nível'] = cefr_counts['Classificação'].apply(group_level)

    # Ensure no 'Outro' is present
    cefr_counts = cefr_counts.dropna(subset=['Grupo Nível'])

    # Step 11: Create the Sunburst Chart
    fig = px.sunburst(
        cefr_counts,
        path=['Grupo Nível', 'Classificação'],
        values='Count',
        color='Grupo Nível',
        color_discrete_map={
            'Nível limitado': 'lightblue',
            'Nível independente': 'lightgreen',
            'Nível competente': 'lightcoral',
            'Nível sólido': 'lightgray'
        },
        title=f'Distribuição de Colaboradores por Nível CEFR - Idioma: {selected_idioma} - Acompanhamento',
        hover_data={'Count': True},
        labels={'Count': 'Número de Colaboradores'}
    )

    # Step 12: Update Layout for Better Appearance
    fig.update_layout(
        margin=dict(t=50, l=25, r=25, b=25)
    )

    return fig

# Step 13: Create an Interactive Dropdown Widget
idioma_dropdown = widgets.Dropdown(
    options=idiomas_sorted,
    value=idiomas_sorted[0],  # Set the default value to the first language
    description='Idiomas:',
    disabled=False,
)

# Step 14: Define the Function to Update the Plot Based on Dropdown Selection
def update_plot(selected_idioma):
    fig = create_sunburst(selected_idioma)
    fig.show()

# Step 15: Use `interactive_output` to Link the Dropdown with the Plot Update Function
out = widgets.interactive_output(update_plot, {'selected_idioma': idioma_dropdown})

# Step 16: Display the Dropdown and the Output Plot
display(idioma_dropdown, out)


Dropdown(description='Idiomas:', options=('idioma inglês',), value='idioma inglês')

Output()